In [37]:
import time
import bitso
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import plt
plt.style.use('seaborn')
%matplotlib inline

In [38]:
scope = ['https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope)
client = gspread.authorize(creds)

Historical = client.open("Historical Bitso")
BTC = Historical.get_worksheet(0)
ETH = Historical.get_worksheet(1)
XRP = Historical.get_worksheet(2)
BCH = Historical.get_worksheet(3)

In [39]:
def get_df(WS):

    col_names=True
    row_names=True

    raw_data = WS.get_all_values()

    start_row_int, start_col_int = WS.get_int_addr('A1')

    rows, cols  = np.shape(raw_data)

    raw_data = [row[start_col_int-1:] for row in raw_data[start_row_int-1:]]

    if row_names and col_names:
        row_names = [row[0] for row in raw_data[1:]]
        col_names = raw_data[0][1:]
        raw_data = [row[1:] for row in raw_data[1:]]
    elif row_names:
        row_names = [row[0] for row in raw_data]
        col_names = np.arange(len(raw_data[0]) - 1)
        raw_data = [row[1:] for row in raw_data]
    elif col_names:
        row_names = np.arange(len(raw_data) - 1)
        col_names = raw_data[0]
        raw_data = raw_data[1:]
    else:
        row_names = np.arange(len(raw_data))
        col_names = np.arange(len(raw_data[0]))

    df = pd.DataFrame([pd.Series(row) for row in raw_data], index=row_names)
    df.columns = col_names
    df = df.convert_objects(convert_numeric=True)
    return df

In [40]:
#Bitso_BTC = get_df(XRP)
#test = Bitso_BTC.convert_objects(convert_numeric=True)
#type(test.ASK[0])

In [41]:
def get_strategy(pm1, pm2, Bitso):
    
    Bitso['PM1'] = Bitso['LAST'].rolling(pm1).mean()
    Bitso['PM2'] = Bitso['LAST'].rolling(pm2).mean()
    Bitso.dropna(inplace = True)
    Bitso = Bitso[['LAST','PM1','PM2']]
    Bitso['Posicion'] = np.where(Bitso['PM1'] > Bitso['PM2'], 1, -1)
    Bitso['Retornos'] = np.log(Bitso['LAST']/Bitso['LAST'].shift(1))
    Bitso.dropna(inplace = True)
    Bitso['Estrategia'] = Bitso['Retornos']* Bitso['Posicion'].shift(1)
    Bitso.dropna(inplace = True)
    Bitso['Retacum'] = Bitso['Retornos'].cumsum().apply(np.exp)
    Bitso['Estracum'] = Bitso['Estrategia'].cumsum().apply(np.exp)

    #------------------Print it out
    '''
    fig = plt.figure(figsize=(14,14))

    ax1 = plt.subplot2grid((25,1),(0,0),rowspan = 8, colspan = 1)
    Bitso['LAST'].plot(title = ('Estrategia PMs para Bitso ' + str(pm1*10) + ', ' + str(pm2*10)))
    Bitso['PM1'].plot(ax=ax1)
    Bitso['PM2'].plot(ax=ax1)

    ax1.set_ylabel('Ultimo Precio, PM1, PM2')
    plt.xticks([],[])

    ax2 = plt.subplot2grid((25,1),(8,0), rowspan = 3, colspan = 1)
    Bitso['Posicion'].plot(ylim=[-1.1,1.1])
    ax2.set_ylabel('Posicion')
    plt.xticks([],[])

    ax3=plt.subplot2grid((25,1),(11,0),rowspan = 8, colspan = 1)
    Bitso[['Retacum', 'Estracum']].plot(ax = ax3)
    ax3.set_ylabel('Retornos Acumulados')
    plt.xticks([],[])
    '''
    
    #----------------------------------------
    
    return Bitso['Estracum'].tail(1).copy()

In [ ]:
def best_strategy(pm1, pm2, Bitso):
    
    Bitso['PM1'] = Bitso['LAST'].rolling(pm1).mean()
    Bitso['PM2'] = Bitso['LAST'].rolling(pm2).mean()
    Bitso.dropna(inplace = True)
    Bitso = Bitso[['LAST','PM1','PM2']]
    Bitso.plot(figsize=(12,8))
    Bitso['Posicion'] = np.where(Bitso['PM1'] > Bitso['PM2'], 1, -1)
    Bitso.plot(figsize=(12,8), secondary_y = 'Posicion')
    Bitso['Retornos'] = np.log(Bitso['LAST']/Bitso['LAST'].shift(1))
    Bitso.dropna(inplace = True)
    Bitso['Estrategia'] = Bitso['Retornos']* Bitso['Posicion'].shift(1)
    Bitso.dropna(inplace = True)
    Bitso['Retacum'] = Bitso['Retornos'].cumsum().apply(np.exp)
    Bitso['Estracum'] = Bitso['Estrategia'].cumsum().apply(np.exp)

    #------------------Print it out
    fig = plt.figure(figsize=(14,14))

    ax1 = plt.subplot2grid((25,1),(0,0),rowspan = 8, colspan = 1)
    Bitso['LAST'].plot(title = ('Estrategia PMs para Bitso ' + str(pm1*10) + ', ' + str(pm2*10)))
    Bitso['PM1'].plot(ax=ax1)
    Bitso['PM2'].plot(ax=ax1)

    ax1.set_ylabel('Ultimo Precio, PM1, PM2')
    plt.xticks([],[])

    ax2 = plt.subplot2grid((25,1),(8,0), rowspan = 3, colspan = 1)
    Bitso['Posicion'].plot(ylim=[-1.1,1.1])
    ax2.set_ylabel('Posicion')
    plt.xticks([],[])

    ax3=plt.subplot2grid((25,1),(11,0),rowspan = 8, colspan = 1)
    Bitso[['Retacum', 'Estracum']].plot(ax = ax3)
    ax3.set_ylabel('Retornos Acumulados')
    plt.xticks([],[])
    
    return Bitso

In [ ]:
Best_Stra = 0
Best_PM1 = 0
Best_PM2 = 0
Bitso_BTC = get_df(ETH)

#For cycle para ver cual es la mejor combinacion de 10 minutos hasta 6 horas

for pm1 in np.arange(1, 144):
    for pm2 in np.arange(1, 144):
        if (pm1<pm2):
            Val_Stra = get_strategy(pm1, pm2, Bitso_BTC.copy())
        
            if (Val_Stra[0] > Best_Stra):
                Best_Stra = Val_Stra[0]
                Best_PM1 = pm1
                Best_PM2 = pm2
                print (pm1,pm2,Best_Stra)
        

C:\Users\Antonio\Anaconda3\envs\Crypto\lib\site-packages\ipykernel_launcher.py:32: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
C:\Users\Antonio\Anaconda3\envs\Crypto\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\Antonio\Anaconda3\envs\Crypto\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Antonio\Anaconda3\envs\Crypto\lib\site-packages\ip

1 2 0.000430474891835
1 77 0.000430563790239
1 78 0.000479582073153
1 79 0.000597347348701
1 80 0.000597930529665
1 81 0.000611374201299
1 82 0.000657008299345
1 83 0.000664962004969
1 84 0.000680761820656
1 85 0.00074199899085
1 86 0.000863393621784
1 87 0.000897262782414
1 88 0.00100504050773
1 89 0.00105546331889
1 90 0.00111362749219
1 91 0.00131753994376
1 92 0.00140154937105
1 93 0.00141860945722
1 94 0.00151360189069
1 95 0.00153971728052
1 96 0.00203591087393
1 97 0.0023379854514
1 98 0.00250064125799
1 99 0.00270262096098
1 100 0.00310537718542
1 101 0.00323058221864
1 102 0.00414885137947
1 103 0.00469743845203
1 104 0.00533417904906
1 105 0.00570984972454
1 106 0.00614523587661
1 107 0.00663726333537
1 108 0.00758618530491
1 110 0.00792258980728
1 111 0.00804296118514
1 112 0.00846593020739
1 113 0.00976956967858
1 115 0.0103564372106
1 116 0.0110549901271
1 117 0.0112052008699
1 118 0.0124450048125
1 119 0.0134311925538
1 120 0.0143136030111
1 122 0.0146416095355
1 123 0.01

In [ ]:
Best = best_strategy(Best_PM1,Best_PM2,get_df(ETH))
#Best.to_csv('best.csv')

In [ ]:
Best['Estracum'].tail(1) - Best['Retacum'].tail(1)